In [ ]:
import scanpy as sc
import numpy as np

# Load preprocessed data (~2K genes)
DATA_DIR = ...

GEX = sc.read_h5ad(DATA_DIR / 'Cite_GEX_with_shared_preprocessed.h5ad')
ADT = sc.read_h5ad(DATA_DIR / 'Cite_ADT_preprocessed.h5ad')

In [ ]:
# Reduce the number of categories in the cell type annotation
reduced_cell_type = []

for cell in GEX.obs["cell_type"]: 
    if "CD8+" in cell: 
        reduced_cell_type.append("CD8+")
    elif "B1" in cell: 
        reduced_cell_type.append("B1")
    elif "CD4+" in cell: 
        reduced_cell_type.append("CD4+")
    elif "Plasma" in cell: 
        reduced_cell_type.append("Plasma cell")
    elif "NK" in cell:
        reduced_cell_type.append("NK")
    elif ("gdT TCRVD2+" in cell) or ("dnT" in cell) or ("cDC1" in cell) or ("T prog cycling" in cell):
        reduced_cell_type.append("Other")
    elif "Naive CD20+ B IGKC" in cell:
        reduced_cell_type.append("Naive CD20+ B IGKC")
    elif "ILC" in cell:
        reduced_cell_type.append("ILC")
    else: 
        reduced_cell_type.append(cell)

reduced_cell_type = np.asarray(reduced_cell_type)

In [ ]:
# linear model to classify into cell types based on factors 
from sklearn.linear_model import LogisticRegression


def linreg_eval(features, labels, test_split):
    """ test_split: bool np array indicating test set """
    X_train, y_train = features[~test_split], labels[~test_split]
    X_test, y_test = features[test_split], labels[test_split]

    clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return clf, y_test, y_pred, np.mean(y_pred == y_test)


In [ ]:
from collections import defaultdict


TRAIN_SIZE = 0.66
num_samples = len(GEX.obs)

# create train/test split, same for all models
np.random.seed(42)
test_split = np.random.randn(num_samples) > TRAIN_SIZE  


# evaluate different models
# TODO: input your Z matrix here
factor_methods = dict(
    random_model = np.ones((num_samples, 10)),
    mofa_model = ...,  # input your Z matrix here (samples x factors)
    #... add more models here
)

model_results = defaultdict(dict)

# evaluate different methods -  classification into cell types, smoker, gender
evaluation_methods = dict(
    cell_type=GEX.obs['cell_type'], 
    reduced_cell_type=reduced_cell_type,
    smoker=GEX.obs['DonorSmoker'],
    gender=GEX.obs['DonorGender'],
)

In [ ]:
# run evaluation
for model_name, feats in factor_methods.items():
    print(model_name)
    for method_name, labels in evaluation_methods.items():
        acc = np.mean(labels[test_split] == np.random.choice(labels))

        clf, y_test, y_pred, acc = linreg_eval(feats, labels, test_split)
        print(f"{method_name} accuracy:", f'{acc:.2f}')
        
        model_results[model_name][method_name] = acc
    print()
